In [ ]:
import pandas as pd
import numpy as np
import pickle
from keras.models import load_model
from utils import S2FPS, S2DES, S2MAC

In [ ]:
test_file = 'test.csv'
model_file = 'rt_prediction_model.h5'
lab_file = 'label_encoder.pkl'
scale_file = 'scaler.pkl'
output_file = 'output1.csv'

In [ ]:
# Read data
data = pd.read_csv(test_file)

# S2 Features
X_fps = np.array([S2FPS(s) for s in data['SMILES']])
X_des = np.array([S2DES(s) for s in data['SMILES']])
X_mac = np.array([S2MAC(s) for s in data['SMILES']])

# Load LabelEncoder
with open(lab_file, 'rb') as f:
    le = pickle.load(f)

# Lab Encoder
lab_encoded = le.transform(data['Lab'])
num_labs = len(le.classes_)
X_lab = np.eye(num_labs)[lab_encoded]

# Load model
model = load_model(model_file)

# Load StandardScaler
with open(scale_file, 'rb') as f:
    scaler = pickle.load(f)

# Prediction
y_pred_scaled = model.predict([X_fps, X_des, X_mac, X_lab])
y_pred = scaler.inverse_transform(y_pred_scaled)
data['RT'] = y_pred

# Data output
data.to_csv(output_file, index=False)